# This notebook will facilitate the end-to-end usage of the NordAxon Code Monkeys solution to the Adipocyte Imaging Challenge

## Depending on your file structure, you will run different commandos.
### These will all be explained below.

If your file structure corresponds to the following (target and mask folders are optional):

- Data dir
    - input
        - 20x
            - img1
            - img2
            - ...
        - 40x
            - ...
        - 60x
            - ...
    - target
        - 20x
        - 40x
        - 60x
    - masks
        - 20x
        - 40x
        - 60x
    
Run the cells below:

In [ ]:
!python print_hello.py --input-dir "path_to_test" --output-dir "path_to_preprocessed_data_dir"

And if your file structure corresponds to this format (target and mask folders still optional):

- Data dir
    - 20x
        - Input
            - img1
            - img2
            - ...
        - Target
            - ...
        - Masks
            - ...
    - 40x
        - Input
        - Target
        - Masks
    - 60x
        - Input
        - Target
        - Masks
    
Run the cells below: